In [1]:
import os
import gffutils
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import json
from pyfaidx import Fasta
from Bio import SeqIO
import re
from pprint import pprint as pp

In [2]:
import pandas
from bokeh.charts import Histogram, show
from bokeh.io import output_notebook

In [3]:
# globals
species_list = ["Bcur", "Bdor", "Bole", "Ccap"]
gff_path = "./input/gff/"
fasta_path = "./input/fasta/"
groups_fn = "./input/groups_filtered_6181genes.txt"
output_path = "./output/"
intermediate_path = "./intermediate/"
aligned_fasta_path = "./input/aligned_4spp_fasta/"

# gap filter parameters
max_gap_percent = 0
max_gap_length = 0
# cds length filter parameters
min_cds_length = 400
max_cds_length = 600

# create handles for all .db files in intermediate directory
gff_fn = {name.split('.gff.db')[0]: intermediate_path + name for name in os.listdir(intermediate_path) if ".gff.db" in name}
gff = {key: gffutils.FeatureDB(value) for key, value in gff_fn.items()}

# create handles for all .fasta files in fasta directory
fasta_fn = {name.split('.nt.fasta')[0]: fasta_path + name for name in os.listdir(fasta_path) if
         ((".nt.fasta" in name) and (".nt.fasta.fai" not in name))}
fasta = {key: Fasta(value) for key, value in fasta_fn.items()}

# import ortholog groups
with open(intermediate_path + "groups.json", 'r') as f:
    parent_groups = json.load(f)

# create handles for all .fasta files in aligned_4spp_fasta directory
aligned_fasta_fn = {name.split('.fasta')[0]: aligned_fasta_path + name for name in os.listdir(aligned_fasta_path) if
         ((".fasta.aln" in name) and (".fasta.aln.fai" not in name))}

In [4]:
# define functions to parse coordinates of cds's from concatinated aligned fasta w/ n's and -'s
nnn = 50
def findBreakpoints(seq):
    breakpoints = []
    loc = 0
    regex = re.compile(r"n+[-+n+]*")
    while(True):
        #loc = seq.find(nnn, loc)
        match = regex.search(seq, loc)
        if not match:
            break
        if len(match.group().replace('-', '')) >= nnn:
            breakpoints.append(match.span())
        loc = match.end()
    return(breakpoints)

def findExonCoords(seq):
    breakpoints = findBreakpoints(seq)
    length = len(seq)

    if len(breakpoints) == 0:
        return([(0, length)])

    if len(breakpoints) == 1:
        bp = breakpoints[0]
        return([(0, bp[0]), (bp[1], length)])

    elif len(breakpoints) > 0:
        exonCoords = []
        exonCoords.append((0, breakpoints[0][0])) # first exon

        for i in range(len(breakpoints) + 1)[1:-1]: # all intermediate exons
            ex_start = breakpoints[i-1][1]
            ex_end = breakpoints[i][0]
            exonCoords.append((ex_start, ex_end))

        exonCoords.append((breakpoints[-1][1], length)) # last exon
        return(exonCoords)
    
def gapPercent(seq):
    seq = str(seq)
    gappedLen = len(seq)
    gapCount = seq.count('-')
    return( (100.0*gapCount)/gappedLen )

def longestGap(seq):
    seq = str(seq)
    gap_regex = re.compile(r"-+")
    gap_list = gap_regex.findall(seq)
    if gap_list:
        return(sorted([len(gap) for gap in gap_list], reverse=True)[0])
    else:
        return(0)

In [5]:
# read and parse fasta files for each species
aligned_fasta = {}
for ortho in aligned_fasta_fn.keys():
    aligned_fasta[ortho] = {seq_record.id : seq_record 
                                      for seq_record in SeqIO.parse(aligned_fasta_fn[ortho],
                                                                    "fasta", alphabet=IUPAC.ambiguous_dna)}

In [6]:
# parse coords from aligned fasta's
coords = {} # coords[ortho][sp] = [coord, ]
for ortho in aligned_fasta:
    coords[ortho] = {}
    for sp in species_list:
        coords[ortho][sp] = findExonCoords(str(aligned_fasta[ortho][sp].seq))

In [7]:
# Filter aligned exons
ortho_coords = {}
coord_index_dict = {}
for ortho in coords:
    ortho_coords[ortho] = {}
    coord_index_dict[ortho] = {}
    for sp in species_list:
        for i in range(len(coords[ortho][sp])):
            coord = coords[ortho][sp][i]

            # filter for length
            start, end = coord
            length = end - start
            if not min_cds_length <= length <= max_cds_length:
                continue
                
            # filter for gap percent
            seq = str(aligned_fasta[ortho][sp].seq[start:end])
            if gapPercent(seq) > max_gap_percent:
                continue
                
            # filter for gap length
            if longestGap(seq) > max_gap_length:
                continue

            # populate coord_index_dict so I can find unique identifing info with ortho,sp,coord later
            if coord not in coord_index_dict[ortho].keys():
                coord_index_dict[ortho][coord] = {}
            coord_index_dict[ortho][coord][sp] = i
            
            # prep to filter for species membership of ortho
            if coord not in ortho_coords[ortho].keys():
                ortho_coords[ortho][coord] = set()
            ortho_coords[ortho][coord].add(sp)

# set of coords per ortho which were represented in all species
universal_ortho_coords = {}
for ortho in ortho_coords:
    for coord in ortho_coords[ortho]:
        sp_set = ortho_coords[ortho][coord]
        if len(sp_set) == len(species_list):
            if ortho not in universal_ortho_coords.keys():
                universal_ortho_coords[ortho] = set()
            universal_ortho_coords[ortho].add(coord)     

In [8]:
data = []
for ortho in sorted(universal_ortho_coords.keys()):
    for coord in sorted(universal_ortho_coords[ortho]):
        start, end = coord
        length = end - start
        data.append((ortho, coord, length))

df = pandas.DataFrame.from_records(data=data, columns=['ortho', 'coord', 'Aligned Exon Length'])
print("from {} orthologs".format(len({x[0] for x in data})))
print(df.describe())

output_notebook()
hist = Histogram(df, values="Aligned Exon Length", title="Aligned Exon Length Histogram")
show(hist)

from 1000 orthologs
       Aligned Exon Length
count          1151.000000
mean            487.096438
std              58.433197
min             400.000000
25%             434.500000
50%             480.000000
75%             539.000000
max             600.000000


Loading BokehJS ...

In [9]:
data = [(ortho, len(universal_ortho_coords[ortho])) for ortho in universal_ortho_coords]
                
df = pandas.DataFrame.from_records(data=data, columns=['ortho', 'exons per ortho'])
print(df.describe())

output_notebook()
hist = Histogram(df, values="exons per ortho", title="exons per ortho Histogram")
show(hist)

       exons per ortho
count      1000.000000
mean          1.151000
std           0.407879
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000


Loading BokehJS ...

In [10]:
fasta_prep = {}
for ortho in universal_ortho_coords:
    fasta_prep[ortho] = {}
    cds_list = {}
    for coord in universal_ortho_coords[ortho]:
        fasta_prep[ortho][coord] = {}
        for sp in species_list:
            if sp not in cds_list.keys():
                parent = gff[sp][parent_groups[ortho][sp]]
                cds_list[sp] = [cds for cds in gff[sp].children(parent, featuretype="CDS", order_by="start")]

            index = coord_index_dict[ortho][coord][sp]
            cds = cds_list[sp][index]
            fasta_prep[ortho][coord][sp] = cds

In [11]:
nnn = Seq("nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn", IUPAC.ambiguous_dna)
for ortho in fasta_prep:
    for coord in fasta_prep[ortho]:
        with open(output_path + ortho + "_" + str(coord[0]) + "-" + str(coord[1]) + ".13spp.fasta", "w") as f:
            for sp in species_list:
                cds = fasta_prep[ortho][coord][sp]
                start,end = coord
                seq = aligned_fasta[ortho][sp].seq[start:end]
                seqReq = SeqRecord(seq, id=sp, description=cds.id)
                f.write(seqReq.format("fasta"))